In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.9 MB/s eta 0:00:00


#데이터 전처리

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Reshape, MultiHeadAttention
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 난수 시드 고정
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

# 시드 고정
set_seed(42)


In [ ]:
# 데이터 경로
train_data_dir = '/content/drive/MyDrive/Data/img/train'
validation_data_dir = '/content/drive/MyDrive/Data/img/val'
test_data_dir = '/content/drive/MyDrive/Data/img/test'

# 데이터 증강
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5853 images belonging to 4 classes.
Found 1168 images belonging to 4 classes.
Found 1137 images belonging to 4 classes.


#MobileNetV1 기반 모델링(실행환경 T4)

In [ ]:
from tensorflow.keras.layers import SeparableConv2D,Conv2D

def squeeze_excite_block(input_tensor, ratio=16):
    init = input_tensor
    channel_axis = 1 if tf.keras.backend.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = tf.keras.layers.multiply([init, se])
    return x

def create_mobilenet_with_attention(input_shape, num_classes):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)

    # 패치 추출 레이어
    patch_extraction = tf.keras.Sequential([
        SeparableConv2D(256, kernel_size=4, strides=4, padding='same', activation='relu'),
        SeparableConv2D(256, kernel_size=2, strides=2, padding='valid', activation='relu'),
        Conv2D(256, kernel_size=1, strides=1, padding='valid', activation='relu')
    ], name='patch_extraction')
    x = patch_extraction(x)
    print("패치 추출 후:", x.shape)  # 텐서 형태 출력
    x = squeeze_excite_block(x)
    print("SE Block 후:", x.shape)  # 텐서 형태 출력
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# 모델 생성 및 컴파일
model = create_mobilenet_with_attention(input_shape=(224, 224, 3), num_classes=4)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])


# 모델 구조 출력 (선택 사항)
model.summary()

패치 추출 후: (None, 1, 1, 256)
SE Block 후: (None, 1, 1, 256)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)           3228864   ['input_6[0][0]']             
 onal)                                                                                            
                                                                                                  
 patch_extraction (Sequenti  (None, 1, 1, 256)            411392    ['mobilenet_1.00_224[0][0]']  
 al)                                                                                              
                                     

In [ ]:
sample_input = np.random.random((1, 224, 224, 3)).astype(np.float32)

model.predict(sample_input)


1/1 [==============================] - 3s 3s/step


array([[0.25012082, 0.24982189, 0.24986042, 0.2501969 ]], dtype=float32)

In [ ]:
# 학습률 조정 콜백
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

# EarlyStopping 콜백
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[reduce_lr, early_stopping])

Epoch 1/100
183/183 [==============================] - 4127s 23s/step - loss: 1.3227 - accuracy: 0.3379 - val_loss: 1.2636 - val_accuracy: 0.3853 - lr: 0.0010
Epoch 2/100
183/183 [==============================] - 452s 2s/step - loss: 1.1748 - accuracy: 0.4681 - val_loss: 1.1920 - val_accuracy: 0.4743 - lr: 0.0010
Epoch 3/100
183/183 [==============================] - 449s 2s/step - loss: 1.1197 - accuracy: 0.5044 - val_loss: 1.2220 - val_accuracy: 0.4812 - lr: 0.0010
Epoch 4/100
183/183 [==============================] - 449s 2s/step - loss: 1.0603 - accuracy: 0.5435 - val_loss: 1.1752 - val_accuracy: 0.4709 - lr: 0.0010
Epoch 5/100
183/183 [==============================] - 446s 2s/step - loss: 1.0246 - accuracy: 0.5628 - val_loss: 1.1820 - val_accuracy: 0.4692 - lr: 0.0010
Epoch 6/100
183/183 [==============================] - 457s 2s/step - loss: 0.9837 - accuracy: 0.5852 - val_loss: 1.1982 - val_accuracy: 0.4675 - lr: 0.0010
Epoch 7/100
183/183 [==============================] - 4

In [ ]:
# 파인튜닝 과정
base_model = model.layers[1]
base_model.trainable = True

# 사전 학습된 모델의 전체 레이어 갯수 가져오기
num_layers = len(base_model.layers)

# 훈련되지 않고 고정될 레이어 비율
freeze_ratio = 0.6
num_freeze_layers = int(num_layers * freeze_ratio)

# 하위 레이어 고정, 상위 레이어 해제
for layer in base_model.layers[:num_freeze_layers]:
    layer.trainable = False
for layer in base_model.layers[num_freeze_layers:]:
    layer.trainable = True

# 모델 재컴파일
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history_fine = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[reduce_lr, early_stopping])

Epoch 1/100
183/183 [==============================] - 450s 2s/step - loss: 1.0965 - accuracy: 0.5201 - val_loss: 0.9983 - val_accuracy: 0.5616 - lr: 1.0000e-04
Epoch 2/100
183/183 [==============================] - 445s 2s/step - loss: 0.9534 - accuracy: 0.5963 - val_loss: 0.9458 - val_accuracy: 0.5899 - lr: 1.0000e-04
Epoch 3/100
183/183 [==============================] - 452s 2s/step - loss: 0.8789 - accuracy: 0.6330 - val_loss: 0.9170 - val_accuracy: 0.6199 - lr: 1.0000e-04
Epoch 4/100
183/183 [==============================] - 458s 3s/step - loss: 0.8209 - accuracy: 0.6586 - val_loss: 0.8082 - val_accuracy: 0.6601 - lr: 1.0000e-04
Epoch 5/100
183/183 [==============================] - 444s 2s/step - loss: 0.7624 - accuracy: 0.6832 - val_loss: 0.7564 - val_accuracy: 0.6807 - lr: 1.0000e-04
Epoch 6/100
183/183 [==============================] - 453s 2s/step - loss: 0.7069 - accuracy: 0.7078 - val_loss: 0.8331 - val_accuracy: 0.6610 - lr: 1.0000e-04
Epoch 7/100
183/183 [=============

In [ ]:
# 검증 데이터 확인
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

37/37 [==============================] - 62s 2s/step - loss: 0.6855 - accuracy: 0.7277
Validation Loss: 0.6855243444442749
Validation Accuracy: 0.7277397513389587


In [11]:
# 테스트 데이터 전체 검증
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


36/36 [==============================] - 489s 14s/step - loss: 0.6840 - accuracy: 0.7291
Test Loss: 0.6839980483055115
Test Accuracy: 0.7291116714477539


In [12]:
# 모델 저장
model.save('/content/drive/MyDrive/MobileNetV1_base.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 결과 그래프 출력
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']

loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, 1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

#임의의 이미지로 모델 성능 test

In [ ]:
# 이미지 전처리 함수
def load_and_prepare_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.0  # 정규화
    return img_tensor

# 감정을 예측하는 함수
def predict_emotion(model, img_path):
    print("이미지 로딩 중...")
    test_image = load_and_prepare_image(img_path)
    print("감정 예측 중...")
    prediction = model.predict(test_image)
    return prediction

# 모델 경로와 이미지 경로
model_path = '/content/my_model_MobileNetV2.h5'  # 학습된 모델 경로
image_path = '/content/drive/MyDrive/test/1.jpg'  # 테스트할 이미지 경로

# 모델 로드
model = load_model(model_path)

# 예측 결과 얻기
predictions = predict_emotion(model, image_path)
emotion_index = np.argmax(predictions)
emotion_labels = ['화남', '행복', '슬픔', '충격']  # 실제 감정 레이블로 변경

print("예측된 감정:", emotion_labels[emotion_index])
print("각 감정의 비율:")
for i, label in enumerate(emotion_labels):
    print(f"{label}: {predictions[0][i] * 100:.2f}%")


이미지 로딩 중...
감정 예측 중...


1/1 [==============================] - 1s 704ms/step
예측된 감정: 행복
각 감정의 비율:
화남: 4.20%
행복: 79.71%
슬픔: 1.21%
충격: 14.88%
